# Crear Target

## Ejercicio 1:
Cree el archivo `competencia_01.csv`, usando el `competencia_01_crudo.csv` con una variable adicional llamada `clase_ternaria`, que contenga las categorías **CONTINUA**, **BAJA+1** y **BAJA+2** 

### Ayudita 

Para practicar el muy útil y necesario lenguaje **SQL**, vamos a utilizar una base de datos **OLAP** llamada **DuckDB**. 

La documentación la puede encontrar [aquí](https://duckdb.org/docs/archive/0.8.1/sql/introduction)
Procedemos a instalarla, esto se debe ejecutar una sola vez

In [1]:
##%%bash 
# ! pip install duckdb
# ! pip install jupysql
# ! pip install duckdb-engine
# ! pip install pandas

  Obtaining dependency information for duckdb from https://files.pythonhosted.org/packages/ee/af/e3fdfa4776124e797d8303f85f3bd97e17f7a1aa1a90189b545369d267d1/duckdb-0.8.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.8 MB 8.6 MB/s eta 0:00:02
   --- ------------------------------------ 0.8/9.8 MB 10.4 MB/s eta 0:00:01
   -------- ------------------------------- 2.1/9.8 MB 16.6 MB/s eta 0:00:01
   -------- ------------------------------- 2.1/9.8 MB 16.6 MB/s eta 0:00:01
   -------- ------------------------------- 2.1/9.8 MB 16.6 MB/s eta 0:00:01
   -------- ------------------------------- 2.1/9.8 MB 16.6 MB/s eta 0:00:01
   -------- ------------------------------- 2.1/9.8 MB 16.6 MB/s eta 0:00:01
   -------- ------------------------------- 2.1/9.8 MB 16.6 MB/s eta 0:00:01
   -------- ------------------------------- 2.1/9.8 MB 16.6 MB/s eta 0:00:01
   -------------- ---------------

Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb** 

In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

Y ya podemos usar **SQL** dentro de una notebook!

In [2]:
%%sql
select 'hola mundo' 

,'hola mundo'
0,hola mundo


Para cargar el archivo `.csv` a una tabla:

In [3]:
%%sql
create or replace table competencia_01 as 
select 
    *
from read_csv_auto('C:\Users\vanes\Documents\UBA\2do_cuatrimestre\DMEyF\datasets\competencia_01_crudo.csv')

,Success


Hagamos unas queries básicas para comprobar que todo esta funcionando bien.

In [4]:
%sql select * from competencia_01 limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,29183981,202103,1,0,0,50,197,14468.81,125765.29,2389.82,...,0.0,7,0.00,-114954.00,0.00,5938,101050.66,68,0,71811.06
1,29184630,202103,1,0,0,59,322,11901.57,74158.93,18750.68,...,0.0,1,0.00,-40330.15,17.59,4089,26834.09,7,0,3894.36
2,29185433,202103,1,0,0,68,268,847.15,21672.47,481.62,...,0.0,21,4692.00,-1173.00,0.00,7829,1651.36,3,0,1560.09
3,29185587,202103,1,0,0,79,322,4976.94,47735.98,1839.31,...,0.0,1,0.00,-15988.67,0.00,7580,30025.29,11,0,1700.85
4,29185646,202103,1,0,0,60,257,2860.45,37800.71,4035.40,...,0.0,21,380616.14,-97383.25,0.00,7827,359610.70,31,0,15600.90


In [5]:
%%sql
select 
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes 
                           -- y lo guarda en un campo llamado cantidad
from competencia_01
group by foto_mes

,foto_mes,cantidad
0,202103,163324
1,202104,163637
2,202105,164102


In [6]:
%%sql
SELECT count(*) FROM competencia_01

,count_star()
0,491063


In [7]:
%%sql
SELECT COUNT(DISTINCT numero_de_cliente) FROM competencia_01

,count(DISTINCT numero_de_cliente)
0,166120


Perfecto, ahora cree una nueva tabla con la variable adicional que se le pide. Algunas funciones que le pueden ser útiles:  [where](https://duckdb.org/docs/sql/query_syntax/where), [left join](https://duckdb.org/docs/sql/query_syntax/from), [case statement](https://duckdb.org/docs/sql/expressions/case)


Con esto creo columna con el primer mes en el que aparece el cliente

In [8]:
%%sql

SELECT numero_de_cliente,
       MIN(foto_mes) OVER (PARTITION BY numero_de_cliente) AS primer_mes
FROM competencia_01

,numero_de_cliente,primer_mes
0,29185646,202103
1,29185646,202103
2,29185646,202103
3,29188883,202103
4,29188883,202103
...,...,...
491058,186316072,202104
491059,186316072,202104
491060,186332374,202104
491061,186332374,202104


Con esto hago un pivot de la tabla a partir del campo foto_mes

In [9]:
%%sql 

WITH base AS (
        SELECT *,
                MIN(foto_mes) OVER (PARTITION BY numero_de_cliente) AS primer_mes              
        FROM competencia_01
    )
    SELECT
        numero_de_cliente,
        primer_mes,
        MAX(CASE WHEN foto_mes = 202103 THEN 1 ELSE 0 END) AS foto_mes_202103,
        MAX(CASE WHEN foto_mes = 202104 THEN 1 ELSE 0 END) AS foto_mes_202104,
        MAX(CASE WHEN foto_mes = 202105 THEN 1 ELSE 0 END) AS foto_mes_202105,
    FROM base
    WHERE numero_de_cliente = 29185646
    GROUP BY numero_de_cliente, primer_mes

,numero_de_cliente,primer_mes,foto_mes_202103,foto_mes_202104,foto_mes_202105
0,29185646,202103,1,1,1


### Query final

Genero la variable target a partir de la vista agg creada:

In [19]:
%%sql 
WITH agg AS (
    WITH min AS (
        SELECT *,
                MIN(foto_mes) OVER (PARTITION BY numero_de_cliente) AS primer_mes              
        FROM competencia_01
    )
    SELECT
        numero_de_cliente,
        primer_mes,
        MAX(CASE WHEN foto_mes = 202103 THEN 1 ELSE 0 END) AS foto_mes_202103,
        MAX(CASE WHEN foto_mes = 202104 THEN 1 ELSE 0 END) AS foto_mes_202104,
        MAX(CASE WHEN foto_mes = 202105 THEN 1 ELSE 0 END) AS foto_mes_202105,
    FROM min
    GROUP BY numero_de_cliente, primer_mes
)
SELECT c.*,
    CASE 
        WHEN (primer_mes = 202103) AND (foto_mes_202104 = 1) AND  (foto_mes_202105 = 1) THEN 'CONTINUA'
        WHEN (primer_mes = 202103) AND (foto_mes_202104 = 0) AND  (foto_mes_202105 = 1) THEN 'CONTINUA' 
        WHEN (primer_mes = 202103) AND (foto_mes_202104 = 0) AND  (foto_mes_202105 = 0) THEN 'BAJA+1' 
        WHEN (primer_mes = 202103) AND (foto_mes_202104 = 1) AND  (foto_mes_202105 = 0) THEN 'BAJA+2'
        WHEN (primer_mes = 202104) AND (foto_mes_202105 = 0)  THEN 'BAJA+1'
        ELSE NULL END AS clase_ternaria         
FROM agg
LEFT JOIN competencia_01 c
ON c.numero_de_cliente = agg.numero_de_cliente

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,29191291,202104,1,0,0,50,146,-67.96,-17272.14,388.83,...,2.0,0.00,-28206.30,1.00,2378.0,6703.17,5.0,0.0,469.20,CONTINUA
1,29246722,202104,1,0,0,65,287,489.52,67704.08,3413.88,...,23.0,66661.59,-101441.04,0.00,8724.0,38624.69,6.0,0.0,66661.59,CONTINUA
2,29292888,202104,1,0,0,46,294,3177.55,6531.26,3293.88,...,9.0,0.00,-37407.01,0.00,5252.0,74727.34,26.0,0.0,11155.23,CONTINUA
3,29316373,202104,1,0,0,62,197,1208.01,8774.18,825.17,...,2.0,0.00,-61371.36,0.00,5490.0,22239.14,14.0,0.0,2205.24,CONTINUA
4,29319942,202104,1,0,0,62,323,472.58,11295.20,6058.90,...,23.0,106273.83,-110971.34,1.16,5441.0,72603.02,34.0,0.0,7636.23,CONTINUA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491058,105753479,202104,1,0,0,35,106,-2447.60,13633.73,800.10,...,9.0,0.00,-17595.00,0.00,3203.0,3331.32,2.0,0.0,3143.64,CONTINUA
491059,127214686,202104,1,0,0,39,80,-266.30,-25258.37,656.51,...,2.0,0.00,-34589.65,0.00,2116.0,1419.70,2.0,0.0,7812.18,CONTINUA
491060,152939313,202104,1,0,0,34,45,1085.36,6754.86,332.83,...,2.0,0.00,-6188.94,0.00,1339.0,1689.12,1.0,0.0,868.02,CONTINUA
491061,152982191,202104,1,0,0,27,44,2973.76,13548.50,1896.77,...,2.0,0.00,-2690.96,14.21,1312.0,1660.96,5.0,0.0,1067.43,CONTINUA


Cantidad de clientes únicos por clase:

In [12]:
%%sql 

WITH final AS (
WITH agg AS (
    WITH base AS (
        SELECT *,
                MIN(foto_mes) OVER (PARTITION BY numero_de_cliente) AS primer_mes              
        FROM competencia_01
    )
    SELECT
        numero_de_cliente,
        primer_mes,
        MAX(CASE WHEN foto_mes = 202103 THEN 1 ELSE 0 END) AS foto_mes_202103,
        MAX(CASE WHEN foto_mes = 202104 THEN 1 ELSE 0 END) AS foto_mes_202104,
        MAX(CASE WHEN foto_mes = 202105 THEN 1 ELSE 0 END) AS foto_mes_202105,
        foto_mes_202103 + foto_mes_202104 + foto_mes_202105 AS suma
    FROM base
    GROUP BY numero_de_cliente, primer_mes
)
SELECT *,
    CASE WHEN (primer_mes = 202103) AND (foto_mes_202104 = 1) AND  (foto_mes_202105 = 1) THEN 'CONTINUA'
         WHEN (primer_mes = 202103) AND (foto_mes_202104 = 0) AND  (foto_mes_202105 = 1) THEN 'CONTINUA' 
         WHEN (primer_mes = 202103) AND (foto_mes_202104 = 0) AND  (foto_mes_202105 = 0) THEN 'BAJA+1' 
         WHEN (primer_mes = 202103) AND (foto_mes_202104 = 1) AND  (foto_mes_202105 = 0) THEN 'BAJA+2'
         WHEN (primer_mes = 202104) AND (foto_mes_202105 = 0)  THEN 'BAJA+1'
         ELSE NULL END AS clase_ternaria         
FROM agg
)
SELECT clase_ternaria, count(*) FROM final
GROUP BY clase_ternaria

,clase_ternaria,count_star()
0,CONTINUA,161312
1,NaN,2790
2,BAJA+2,963
3,BAJA+1,1055


## Ejercicio 1.1

* ¿Cuál es la nominalidad de cada clase?
* ¿Cuál es la proporción del target?

#### Nominalidad:

In [15]:
%%sql 
WITH final AS (
WITH agg AS (
    WITH base AS (
        SELECT *,
                MIN(foto_mes) OVER (PARTITION BY numero_de_cliente) AS primer_mes              
        FROM competencia_01
    )
    SELECT
        numero_de_cliente,
        primer_mes,
        MAX(CASE WHEN foto_mes = 202103 THEN 1 ELSE 0 END) AS foto_mes_202103,
        MAX(CASE WHEN foto_mes = 202104 THEN 1 ELSE 0 END) AS foto_mes_202104,
        MAX(CASE WHEN foto_mes = 202105 THEN 1 ELSE 0 END) AS foto_mes_202105,
        foto_mes_202103 + foto_mes_202104 + foto_mes_202105 AS suma
    FROM base
    GROUP BY numero_de_cliente, primer_mes
)
SELECT agg.numero_de_cliente, c.*,
    CASE WHEN (primer_mes = 202103) AND (foto_mes_202104 = 1) AND  (foto_mes_202105 = 1) THEN 'CONTINUA'
         WHEN (primer_mes = 202103) AND (foto_mes_202104 = 0) AND  (foto_mes_202105 = 1) THEN 'CONTINUA' 
         WHEN (primer_mes = 202103) AND (foto_mes_202104 = 0) AND  (foto_mes_202105 = 0) THEN 'BAJA+1' 
         WHEN (primer_mes = 202103) AND (foto_mes_202104 = 1) AND  (foto_mes_202105 = 0) THEN 'BAJA+2'
         WHEN (primer_mes = 202104) AND (foto_mes_202105 = 0)  THEN 'BAJA+1'
         ELSE NULL END AS clase_ternaria         
FROM agg
INNER JOIN competencia_01 c
ON c.numero_de_cliente = agg.numero_de_cliente
)
SELECT clase_ternaria, count(*) as nominalidad FROM final
GROUP BY clase_ternaria

,clase_ternaria,count_star()
0,CONTINUA,483935
1,BAJA+2,1926
2,NaN,4147
3,BAJA+1,1055


#### Proporción:


In [18]:
%%sql 
WITH final AS (
WITH agg AS (
    WITH base AS (
        SELECT *,
                MIN(foto_mes) OVER (PARTITION BY numero_de_cliente) AS primer_mes              
        FROM competencia_01
    )
    SELECT
        numero_de_cliente,
        primer_mes,
        MAX(CASE WHEN foto_mes = 202103 THEN 1 ELSE 0 END) AS foto_mes_202103,
        MAX(CASE WHEN foto_mes = 202104 THEN 1 ELSE 0 END) AS foto_mes_202104,
        MAX(CASE WHEN foto_mes = 202105 THEN 1 ELSE 0 END) AS foto_mes_202105,
        foto_mes_202103 + foto_mes_202104 + foto_mes_202105 AS suma
    FROM base
    GROUP BY numero_de_cliente, primer_mes
)
SELECT agg.numero_de_cliente, c.*,
    CASE WHEN (primer_mes = 202103) AND (foto_mes_202104 = 1) AND  (foto_mes_202105 = 1) THEN 'CONTINUA'
         WHEN (primer_mes = 202103) AND (foto_mes_202104 = 0) AND  (foto_mes_202105 = 1) THEN 'CONTINUA' 
         WHEN (primer_mes = 202103) AND (foto_mes_202104 = 0) AND  (foto_mes_202105 = 0) THEN 'BAJA+1' 
         WHEN (primer_mes = 202103) AND (foto_mes_202104 = 1) AND  (foto_mes_202105 = 0) THEN 'BAJA+2'
         WHEN (primer_mes = 202104) AND (foto_mes_202105 = 0)  THEN 'BAJA+1'
         ELSE NULL END AS clase_ternaria         
FROM agg
INNER JOIN competencia_01 c
ON c.numero_de_cliente = agg.numero_de_cliente
)
SELECT clase_ternaria, count(*) / 491063 as proporcion FROM final
GROUP BY clase_ternaria

,clase_ternaria,proporcion
0,CONTINUA,0.985485
1,BAJA+2,0.003922
2,BAJA+1,0.002148
3,NaN,0.008445


Para guardar a un **.csv** simplemente debe ejecutar la siguiente sentencia

In [22]:
%%sql
COPY 
    (WITH agg AS (
        WITH min AS (
            SELECT *,
                    MIN(foto_mes) OVER (PARTITION BY numero_de_cliente) AS primer_mes              
            FROM competencia_01
        )
        SELECT
            numero_de_cliente,
            primer_mes,
            MAX(CASE WHEN foto_mes = 202103 THEN 1 ELSE 0 END) AS foto_mes_202103,
            MAX(CASE WHEN foto_mes = 202104 THEN 1 ELSE 0 END) AS foto_mes_202104,
            MAX(CASE WHEN foto_mes = 202105 THEN 1 ELSE 0 END) AS foto_mes_202105,
        FROM min
        GROUP BY numero_de_cliente, primer_mes
    )
    SELECT c.*,
        CASE 
            WHEN (primer_mes = 202103) AND (foto_mes_202104 = 1) AND  (foto_mes_202105 = 1) THEN 'CONTINUA'
            WHEN (primer_mes = 202103) AND (foto_mes_202104 = 0) AND  (foto_mes_202105 = 1) THEN 'CONTINUA' 
            WHEN (primer_mes = 202103) AND (foto_mes_202104 = 0) AND  (foto_mes_202105 = 0) THEN 'BAJA+1' 
            WHEN (primer_mes = 202103) AND (foto_mes_202104 = 1) AND  (foto_mes_202105 = 0) THEN 'BAJA+2'
            WHEN (primer_mes = 202104) AND (foto_mes_202105 = 0)  THEN 'BAJA+1'
            ELSE NULL END AS clase_ternaria         
    FROM agg
    LEFT JOIN competencia_01 c
    ON c.numero_de_cliente = agg.numero_de_cliente)
TO 'C:\Users\vanes\Documents\UBA\2do_cuatrimestre\DMEyF\datasets\competencia_01.csv' (FORMAT CSV, HEADER)

,Success


## Ejercicio 2 - Avanzado
Use ahora el archivo `ejercicio_target.csv` y calcule para todos los clientes en todos los periodos su **clase_ternaria** al mismo tiempo